In [1]:
# https://platform.olimpiada-ai.ro/problems/44

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [6]:
train = pd.read_csv("/kaggle/input/movie-score-prediction/train.csv")
test = pd.read_csv("/kaggle/input/movie-score-prediction/test.csv")

train.drop(columns=['Type'], inplace=True)
test.drop(columns=['Type'], inplace=True)

min_year = min(train['Year'].min(), test['Year'].min())
train['Year'] = train['Year'] - min_year
test['Year'] = test['Year'] - min_year

In [11]:
from sklearn.model_selection import train_test_split
from catboost import Pool

features = ['Title', 'Description', 'Year']
text_features = ['Title', 'Description']

X, y = train[features], train['Score']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.1)

train_pool = Pool(X_train, y_train, text_features=text_features)
valid_pool = Pool(X_valid, y_valid, text_features=text_features)

In [18]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    iterations=1000,
    loss_function='MAE',
    eval_metric='MAE',
    metric_period=25,
    max_depth=8,
)

model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.8084768	test: 0.7781865	best: 0.7781865 (0)	total: 549ms	remaining: 9m 8s
25:	learn: 0.7819076	test: 0.7572931	best: 0.7572931 (25)	total: 13.8s	remaining: 8m 38s
50:	learn: 0.7660974	test: 0.7491379	best: 0.7491379 (50)	total: 25.6s	remaining: 7m 55s
75:	learn: 0.7516216	test: 0.7452019	best: 0.7452019 (75)	total: 37.4s	remaining: 7m 34s
100:	learn: 0.7376278	test: 0.7422357	best: 0.7422357 (100)	total: 49.2s	remaining: 7m 17s
125:	learn: 0.7260635	test: 0.7383258	best: 0.7383258 (125)	total: 1m	remaining: 7m 2s
150:	learn: 0.7143165	test: 0.7374399	best: 0.7374399 (150)	total: 1m 12s	remaining: 6m 48s
175:	learn: 0.7030702	test: 0.7342734	best: 0.7342734 (175)	total: 1m 24s	remaining: 6m 33s
200:	learn: 0.6916972	test: 0.7315655	best: 0.7315655 (200)	total: 1m 35s	remaining: 6m 20s
225:	learn: 0.6807731	test: 0.7290935	best: 0.7290935 (225)	total: 1m 47s	remaining: 6m 8s
250:	learn: 0.6686182	test: 0.7265302	best: 0.7265302 (250)	total: 1m 59s	remaining: 5m 57s
275:	learn

In [24]:
X_test = test[features]

preds = model.predict(X_test)

subm = pd.DataFrame({
    'SampleID': test['SampleID'],
    'Score': preds
})

subm

,SampleID,Score
0,tt13321232,6.808930
1,tt10487072,7.098024
2,tt8004578,7.081704
3,tt11822998,7.195594
4,tt10348700,6.682467
...,...,...
315,tt9581782,6.792327
316,tt7971476,6.692428
317,tt6626676,7.340458
318,tt5545584,7.079507


In [25]:
subm.to_csv("submission.csv", index=False)